In [1]:
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

In [2]:
doctor_cnn = load_model("doctor_cnn_model.keras")
nurse_cnn = load_model("nurse_cnn_model.keras")
staff_cnn = load_model("staff_cnn_model.keras")
patient_cnn = load_model("patient_cnn_model.keras")

C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [3]:
df = pd.read_csv("preprocessed_data_df.csv")

In [4]:
# Define and fit scalers
doctor_scaler = StandardScaler()
nurse_scaler = StandardScaler()
staff_scaler = StandardScaler()
patient_scaler = StandardScaler()

In [5]:
doctor_features = df[[col for col in df.columns if "Doctors" in col]]
nurse_features = df[[col for col in df.columns if "Nurses" in col]]
staff_features = df[['Patients who "Agree" that staff took their preferences into account',
       'Patients who "Disagree" or "Strongly Disagree" that staff took their preferences into account',
       'Patients who "Strongly Agree" that staff took their preferences into account',
       'Staff "always" explained', 'Staff "always" explained new medications',
       'Staff "always" explained possible side effects',
       'Staff "sometimes" or "never" explained',
       'Staff "sometimes" or "never" explained new medications',
       'Staff "sometimes" or "never" explained possible side effects',
       'Staff "usually" explained',
       'Staff "usually" explained new medications',
       'Staff "usually" explained possible side effects']]
patient_features = df[['"Always" quiet at night',
       '"NO", patients would not recommend the hospital (they probably would not or definitely would not recommend it)',
       '"Sometimes" or "never" quiet at night', '"Usually" quiet at night',
       '"YES", patients would definitely recommend the hospital',
       '"YES", patients would probably recommend the hospital']]

# Assume scaler was used and saved separately for each feature group
doctor_scaled = doctor_scaler.fit_transform(doctor_features)
nurse_scaled = nurse_scaler.fit_transform(nurse_features)
staff_scaled = staff_scaler.fit_transform(staff_features)
patient_scaled = patient_scaler.fit_transform(patient_features)

# Reshape for CNN input (samples, timesteps, 1)
doctor_input = doctor_scaled.reshape((doctor_scaled.shape[0], doctor_scaled.shape[1], 1))
nurse_input = nurse_scaled.reshape((nurse_scaled.shape[0], nurse_scaled.shape[1], 1))
staff_input = staff_scaled.reshape((staff_scaled.shape[0], staff_scaled.shape[1], 1))
patient_input = patient_scaled.reshape((patient_scaled.shape[0], patient_scaled.shape[1], 1))


In [6]:
doctor_output = doctor_cnn.predict(doctor_input)
nurse_output = nurse_cnn.predict(nurse_input)
staff_output = staff_cnn.predict(staff_input)
patient_output = patient_cnn.predict(patient_input)


122/122 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [7]:


# Input shape matches the number of features in your tabular data
input_tensor = Input(shape=(12,))
x = Dense(64, activation='relu')(input_tensor)
x = Dense(32, activation='relu')(x)  # This could be your embedding layer
output_tensor = Dense(1, activation='sigmoid')(x)  # Or softmax for classification

input_layer = Input(shape=(6,))
x = Dense(64, activation='relu')(input_layer)
x = Dense(32, activation='relu')(x)
output_layer = Dense(1, activation='sigmoid')(x)




doctor_cnn = Model(inputs=input_tensor, outputs=output_tensor)
nurse_cnn = Model(inputs=input_tensor, outputs=output_tensor)
patient_cnn = Model(inputs=input_layer, outputs=output_layer)
staff_cnn = Model(inputs=input_tensor, outputs=output_tensor)
##Use feature vectors from intermediate layers (better if you want richer features)

doctor_embedding_model = Model(inputs=doctor_cnn.input, outputs=doctor_cnn.layers[-2].output)
nurse_embedding_model = Model(inputs = nurse_cnn.input, outputs = nurse_cnn.layers[-2].output)
staff_embedding_model = Model(inputs= staff_cnn.input, outputs= staff_cnn.layers[-2].output)
patient_embedding_model = Model(inputs = patient_cnn.input, outputs = patient_cnn.layers[-2].output)



In [8]:

doctor_output = doctor_embedding_model.predict(doctor_input)
nurse_output = nurse_embedding_model.predict(nurse_input)
staff_output = staff_embedding_model.predict(staff_input)
patient_output = patient_embedding_model.predict(patient_input)



122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [9]:

combined_features = np.concatenate([doctor_output, nurse_output, staff_output, patient_output], axis=1)


In [10]:
df["Total Ratings"] = (
    df['Patients who gave a rating of "6" or lower (low)'] +
    df['Patients who gave a rating of "7" or "8" (medium)'] +
    df['Patients who gave a rating of "9" or "10" (high)']
)

# Weighted average: 1 for low, 2 for medium, 3 for high
df["Rating"] = (
    1*df['Patients who gave a rating of "6" or lower (low)'] +
    2*df['Patients who gave a rating of "7" or "8" (medium)'] +
    3*df['Patients who gave a rating of "9" or "10" (high)']
) / df["Total Ratings"]


df["Rating"] = df["Rating"] * (5 / 3)
df['Rating'] = df['Rating'].round(2)

In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

y = df["Rating"]  # Target

X_train, X_test, y_train, y_test = train_test_split(combined_features, y, test_size=0.2, random_state=42)

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Evaluate
y_pred = rf_model.predict(X_test)
print("MSE:", mean_squared_error(y_test, y_pred))
print("R²:", r2_score(y_test, y_pred))


MSE: 0.005440349716859724
R²: 0.9143850771437451


In [12]:
import joblib

#joblib.dump(rf_model, "combined_rf_model.pkl")


In [13]:
import joblib
import pandas as pd
import numpy as np
import tensorflow as tf
rf_model = joblib.load("combined_rf_model.pkl")

def predict_rating(hcahps_input_df):
    # Step 1: Load scalers and CNN models
    doctor_scaler = joblib.load("doctor_scaler.pkl")
    nurse_scaler = joblib.load("nurse_scaler.pkl")
    staff_scaler = joblib.load("staff_scaler.pkl")
    patient_scaler = joblib.load("patient_scaler.pkl")

    doctor_cnn = tf.keras.models.load_model("doctor_cnn_model.h5")
    nurse_cnn = tf.keras.models.load_model("nurse_cnn_model.h5")
    staff_cnn = tf.keras.models.load_model("staff_cnn_model.h5")
    patient_cnn = tf.keras.models.load_model("patient_cnn_model.h5")

    rf_model = joblib.load("combined_rf_model.pkl")

    # Step 2: Extract and scale features
    doctor_features = doctor_scaler.transform(hcahps_input_df[[col for col in hcahps_input_df.columns if "Doctor" in col]])
    nurse_features = nurse_scaler.transform(hcahps_input_df[[col for col in hcahps_input_df.columns if "Nurse" in col]])
    staff_features = staff_scaler.transform(hcahps_input_df[[col for col in hcahps_input_df.columns if "Staff" in col]])
    patient_features = patient_scaler.transform(hcahps_input_df[[col for col in hcahps_input_df.columns if "Patient" in col and "rating" not in col.lower()]])

    # Step 3: Reshape for CNNs
    doctor_input = doctor_features.reshape((doctor_features.shape[0], doctor_features.shape[1], 1))
    nurse_input = nurse_features.reshape((nurse_features.shape[0], nurse_features.shape[1], 1))
    staff_input = staff_features.reshape((staff_features.shape[0], staff_features.shape[1], 1))
    patient_input = patient_features.reshape((patient_features.shape[0], patient_features.shape[1], 1))

    # Step 4: CNN outputs
    doctor_output = doctor_cnn.predict(doctor_input)
    nurse_output = nurse_cnn.predict(nurse_input)
    staff_output = staff_cnn.predict(staff_input)
    patient_output = patient_cnn.predict(patient_input)

    # Step 5: Combine CNN outputs as features
    combined_features = np.concatenate([doctor_output, nurse_output, staff_output, patient_output], axis=1)

    # Step 6: Final prediction using Random Forest
    final_prediction = rf_model.predict(combined_features)
    return final_prediction



